Model - DistillBERT tokenization, ADASYN oversampling and HAN model Implemented

In [1]:
pip install transformers



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install imbalanced-learn --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 456.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.set_proxy('http://proxy.example.com:3128', ('USERNAME', 'PASSWORD'))
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [6]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import ADASYN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.layers import Attention, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import RepeatVector

# # Mount Google Drive to access files
# from google.colab import drive
# drive.mount('/content/drive')

# Load CSV dataset
# data = pd.read_csv('/content/drive/MyDrive/Datasets/Eclipse_dataset_new1.csv')
data = pd.read_csv('./Eclipse_dataset_new1_small.csv')

# Count the number of samples for each class
class_counts = data['Priority'].value_counts()

# Find the class with the minimum number of samples
minority_class = class_counts.idxmin()

print("Minority class:", minority_class)
print(class_counts)


# Text preprocessing steps
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase conversion
        text = text.lower()

        # Remove punctuation and special characters
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = text.split()
        tokens = [word for word in tokens if word not in stop_words]
        text = ' '.join(tokens)

        # Stemming
        stemmer = PorterStemmer()
        tokens = text.split()
        tokens = [stemmer.stem(word) for word in tokens]
        text = ' '.join(tokens)

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = text.split()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        text = ' '.join(tokens)

    return text

# Apply text preprocessing to 'description' and 'title' columns
data['Description'] = data['Description'].apply(preprocess_text)
data['Title'] = data['Title'].apply(preprocess_text)

# Tokenize using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_sequence_length = 3000

def tokenize_text(text):
    if isinstance(text, (str, int, float)):
        text = str(text)  # Convert text to string if it's not already

        tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_sequence_length, padding='max_length', truncation=True)
        input_ids = tokens['input_ids']
        attention_mask = tokens['attention_mask']
    else:
        input_ids = []
        attention_mask = []
    return input_ids, attention_mask

# Apply tokenization to 'Description' and 'Title' columns
data['Description'] = data['Description'].apply(tokenize_text)
data['Title'] = data['Title'].apply(tokenize_text)

# Convert tokenized data to numpy arrays
input_ids = np.array([item[0] for item in data['Description']])
attention_masks = np.array([item[1] for item in data['Description']])

# Reshape input IDs and attention masks to have at most 2 dimensions
input_ids = input_ids.reshape(input_ids.shape[0], -1)
attention_masks = attention_masks.reshape(attention_masks.shape[0], -1)

# Concatenate input IDs and attention masks
X_input = np.concatenate((input_ids, attention_masks), axis=1)
y_input = data['Priority']

# Apply ADASYN oversampling
adasyn = ADASYN()
X_resampled, y_resampled = adasyn.fit_resample(X_input, y_input)

# Separate the resampled input IDs and attention masks
input_ids_resampled = X_resampled[:, :max_sequence_length]
attention_masks_resampled = X_resampled[:, max_sequence_length:]

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(input_ids_resampled, y_resampled, test_size=0.2, random_state=42)

# Pad sequences
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

from sklearn.preprocessing import LabelEncoder

# Encode string labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to categorical
num_classes = len(label_encoder.classes_)
y_train = to_categorical(y_train_encoded, num_classes)
y_test = to_categorical(y_test_encoded, num_classes)

# Build HAN model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(len(tokenizer.get_vocab()) + 1, 128)(input_layer)
word_encoder = Bidirectional(GRU(64, return_sequences=True))(embedding_layer)
word_attention = Attention()([word_encoder, word_encoder])
word_flatten = Flatten()(word_attention)
word_dense = Dense(64)(word_flatten)
word_repeat = RepeatVector(max_sequence_length)(word_dense)
sentence_encoder = Bidirectional(GRU(64, return_sequences=True))(word_repeat)
sentence_attention = Attention()([sentence_encoder, sentence_encoder])
sentence_flatten = Flatten()(sentence_attention)
sentence_dense = Dense(64)(sentence_flatten)
output_layer = Dense(num_classes, activation='softmax')(sentence_dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


Minority class: P2
P3    712
P5    152
P4     64
P1     39
P2     33
Name: Priority, dtype: int64
Epoch 1/10


2023-07-10 08:08:41.154451: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-07-10 08:08:41.155416: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 5

89/89 [==============================] - ETA: 0s - loss: 309.9894 - accuracy: 0.2058 

2023-07-10 08:31:41.197754: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-07-10 08:31:41.198225: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 5

89/89 [==============================] - 1482s 17s/step - loss: 309.9894 - accuracy: 0.2058 - val_loss: 196.2365 - val_accuracy: 0.1871
Epoch 2/10
49/89 [===============>..............] - ETA: 9:46 - loss: 300.1740 - accuracy: 0.2098 